In [2]:
import wget

print('Beginning file download with wget module')

url = 'https://raw.githubusercontent.com/Azure/azureml-examples/main/python-sdk/tutorials/automl-with-azureml/image-object-detection/coco2jsonl.py'
wget.download(url, 'coco2jsonl.py')

#The Coco2jsonl.py file is referred from https://github.com/Azure/azureml-examples/blob/main/python-sdk/tutorials/automl-with-azureml/image-object-detection/coco2jsonl.py

Beginning file download with wget module


'coco2jsonl.py'

In [14]:
#Convert COCO dataset to JSONL supported by AzureML for AutoML Model training.
!python coco2jsonl.py \
--input_coco_file_path "./CustomVisionExport_Coco.json" \
--output_dir "./Coco/" --output_file_name "CustomVisionExport_Coco.jsonl" \
--task_type "ObjectDetection" \
--base_url "AmlDatastore://workspaceblobstore/customvisionImages/images/"

Converting for ObjectDetection
Conversion completed. Converted 40 lines.


In [10]:
#Uploading and Registering the Dataset with AzureML 
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
ws = Workspace.get(name='<workspace-name>',
            subscription_id='<sub-id>',
            resource_group='<resource-group>',
            location='<location>',
            cloud='AzureCloud',
            auth=interactive_auth
            )


In [18]:
# Retrieving default datastore that got automatically created when we setup a workspace
ds = ws.get_default_datastore()
ds.upload(src_dir="./Coco//", target_path="customvisionImages/images")

Uploading an estimated of 42 files
Uploading C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\0.jpg
Uploaded C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\0.jpg, 1 files out of an estimated total of 42
Uploading C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\1.jpg
Uploaded C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\1.jpg, 2 files out of an estimated total of 42
Uploading C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\10.jpg
Uploaded C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\10.jpg, 3 files out of an estimated total of 42
Uploading C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\11.jpg
Uploaded C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\11.jpg, 4 files out of an estimated total of 42
Uploading C://Users//varmag//Documents//MyProjects//coco2CustomVision//Coco\12.jpg
Uploaded C://Users//varmag//Documents//MyProjects//coco2Custom

$AZUREML_DATAREFERENCE_254aabd494ed476fbbc27aa5e8073932

In [21]:
# pip install azureml-dataset-runtime --upgrade
from azureml.core import Dataset
from azureml.data import DataType

# get existing training dataset
training_dataset_name = "CVSImagesTrainingDataset"
if training_dataset_name in ws.datasets:
    training_dataset = ws.datasets.get(training_dataset_name)
    print("Found the training dataset", training_dataset_name)
else:
    # create training dataset
    training_dataset = Dataset.Tabular.from_json_lines_files(
        path=ds.path("customvisionImages/images/CustomVisionExport_Coco.jsonl"),
        set_column_types={"image_url": DataType.to_stream(ds.workspace)},
    )
    training_dataset = training_dataset.register(
        workspace=ws, name=training_dataset_name
    )

2022-02-17 09:58:26.032499 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=1245.44 [ms], Info = {'activity_id': 'f7c5e446-1378-4743-a3c7-a6fee50b2ff6', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.38.0.post2', 'dataprepVersion': '', 'sparkVersion': '3.2.1', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': '', 'completionStatus': 'Success', 'durationMs': 0.0}, Exception=UserErrorException; UserErrorException:
	Message: Cannot find dataset registered with name "CVSImagesTrainingDataset" (version: None) in the workspace.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot find dataset registered with name \"CVSImagesTrainingDataset\" (version: None) in the workspace."
    }
}
